<a href="https://colab.research.google.com/github/bl-young/USEEIO_training/blob/main/Notebooks/useeior.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# useeior

`useeior` is an R package for building and using [USEEIO models](https://www.epa.gov/land-research/us-environmentally-extended-input-output-useeio-models).

The [model object](format_specs/Model.md) is the primary output that is built according to a given [model specification](format_specs/ModelSpecification.md) and optional hybridization specification, e.g. [disaggregation](format_specs/ModelCustomization.md).
[Model specifications](inst/extdata/modelspecs) and associated hybridization specifications, e.g. [disaggregation](inst/extdata/disaggspecs), for EPA-validated models are included in the package.

`useeior` offers various functions for validating, calculating, visualizing, and writing out models and/or their components.
`useeior` is a core component of the [USEEIO Modeling Framework](https://github.com/USEPA/useeio) and is in a stable development state.
Users intending to use the package for production purposes and applications should use [Releases](https://github.com/USEPA/useeior/releases).
`useeior` v1.0.0 was peer-reviewed internally at USEPA and published at Zenodo.

## Installing useeior and building a model

In [1]:
# Clones the repository into your local colab environment
system("git clone https://github.com/USEPA/useeior.git")

In [ ]:
# Install the package using devtools, this will take 5-6 minutes
install.packages("devtools")
devtools::install("useeior/.")
# Because we want access to the repository in Google Colab we are installing it locally.
# Otherwise you could use:
# devtools::install_github("USEPA/useeior")
library("useeior")

In [ ]:
seeAvailableModels()

In [ ]:
model <- buildModel("USEEIOv2.3-GHG")

## Calculate Results
USEEIO models can be used to calculate national results based on a few parameters:
- Perspective: Can be `DIRECT` or `FINAL` and indicates where the emissions are attributed. In `DIRECT` they are aligned with the sectors that generate the emissions, in `FINAL` they are aligned with the sectors that are being consumed.
- Demand: Can be `Consumption` or `Production`. `Consumption` demand vectors represent total U.S. Final Consumption from households and investment, while `Production` represents total US output whether or not its consumed in the U.S. Alternatively, you can pass a custom demand vector in the form of a named vector.

The [result object](https://github.com/USEPA/useeior/blob/master/format_specs/Calculation.md) has two parts:
- "G": Direct and indirect LCI
- "H": Direct and indirect LCIA (optional)

In [ ]:
result <- calculateEEIOModel(model,
                             perspective = "DIRECT",
                             demand = "Consumption",
                             use_domestic_requirements = FALSE)
print(result$H_r)

In [ ]:
names(model)
write.csv(model$Commodities, 'commodities.csv')
writeModeltoXLSX(model, ".")

In [ ]:
model$Indicators$meta$Name
sector <- "325211/US"
model$Commodities[model$Commodities$Code_Loc == sector,]
contrib <- useeior:::calculateSectorContributiontoImpact(model, sector, "Greenhouse Gases")
contrib$contribution <- scales::percent(contrib$contribution, accuracy = 0.1)
head(subset(contrib, select="contribution"), 5)

## Customizing Models
Download a model spec from useeior/inst/extdata/modelspecs.
Edit and manipulate it and save it as a new file, e.g., `My-model.yml`.
Upload to Colab to a `my_models` folder at the same level as the working directory.

In [ ]:
# Use configpaths to point to a model in a different directory
model <- buildModel("My-model", configpaths = "my_models/My-model.yml")
